<a href="https://colab.research.google.com/github/1FIZANOOR/1FIZANOOR/blob/main/colab_sdxl_turbo_streamlit.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# SDXL-Turbo in Colab

- Make sure to connect to a GPU instance for maximum performance
- Run all cells

In [1]:
!pip install -q streamlit diffusers accelerate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.4/8.4 MB 29.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 47.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 265.7/265.7 kB 32.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.6/190.6 kB 24.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.8/4.8 MB 62.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.1/82.1 kB 11.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 6.1 MB/s eta 0:00:00


In [2]:
!npm install localtunnel

npm WARN saveError ENOENT: no such file or directory, open '/content/package.json'
npm notice created a lockfile as package-lock.json. You should commit this file.
npm WARN enoent ENOENT: no such file or directory, open '/content/package.json'
npm WARN content No description
npm WARN content No repository field.
npm WARN content No README data
npm WARN content No license field.

+ localtunnel@2.0.2
added 22 packages from 22 contributors and audited 22 packages in 1.735s

3 packages are looking for funding
  run `npm fund` for details

found 1 moderate severity vulnerability
  run `npm audit fix` to fix them, or `npm audit` for details


In [3]:
%%writefile app.py
import streamlit as st
from diffusers import AutoPipelineForText2Image
import torch
import os
import time
import random

st.title("SDXL Turbo")

if not os.path.exists('outputs'):
    os.makedirs('outputs')

@st.cache_resource
def load_model():
    device = "cuda" if torch.cuda.is_available() else "cpu"
    pipe = AutoPipelineForText2Image.from_pretrained("stabilityai/sdxl-turbo", torch_dtype=torch.float16, variant="fp16", safety_checker=None).to(device)
    # pipe.enable_freeu(s1=0.9, s2=0.2, b1=1.3, b2=1.4)
    return pipe

def realtime_gen():
    with st.spinner('Generating image...'):
        # Generate the image
        image = pipe(prompt=prompt, negative_prompt=negative_prompt, num_inference_steps=num_inference_steps, guidance_scale=guidance_scale).images[0]

        # Display the image
        st.image(image, caption='Generated Image')

pipe = load_model()
realtime = st.sidebar.checkbox('Realtime')
prompt = st.sidebar.text_area("Enter a prompt:", "A cinematic shot of a baby raccoon wearing an intricate Italian priest robe.", height=1, on_change=realtime_gen if realtime else None)
negative_prompt = st.sidebar.text_area("Enter the negative prompt:", "low resolution, grainy, distorted", height=1)
num_inference_steps = st.sidebar.slider("Number of inference steps", 1, 10, 1)
guidance_scale = st.sidebar.slider("Guidance scale", 0.0, 2.0, 1.0)

if st.sidebar.button('Generate Image'):
    with st.spinner('Generating image...'):
        # Generate the image
        image = pipe(prompt=prompt, negative_prompt=negative_prompt, num_inference_steps=num_inference_steps, guidance_scale=guidance_scale).images[0]

        # Generate a unique filename
        timestamp = int(time.time())
        random_number = random.randint(0, 99999)
        filename = f"outputs/image_{timestamp}_{random_number}.png"

        # Save the image
        image.save(filename)

        # Display the image
        st.image(filename, caption='Generated Image')

Writing app.py


In [ ]:
# Random port
!echo $((8000 + $RANDOM % 1000)) > /content/PORT.txt

!echo "Open the url, paste the below IP address in Endpoint IP field and off you go..."

# Run Streamlit app, redirect output to logs.txt, expose using localtunnel using random port, and get public IP
!streamlit run /content/app.py --server.port $(</content/PORT.txt) &> /content/logs.txt & npx localtunnel --port $(</content/PORT.txt) & curl ipv4.icanhazip.com

Open the url, paste the below IP address in Endpoint IP field and off you go...
35.199.188.107
npx: installed 22 in 2.106s
your url is: https://huge-crabs-enter.loca.lt
